In [ ]:
# import library
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# load the dataset what has been taken by kaggle
movies=pd.read_csv("/content/tmdb_5000_movies.csv")
credits=pd.read_csv("/content/tmdb_5000_credits.csv", engine='python', on_bad_lines='warn')

/tmp/ipython-input-1019443308.py:2: ParserWarning: Skipping line 848: unexpected end of data

  credits=pd.read_csv("/content/tmdb_5000_credits.csv", engine='python', on_bad_lines='warn')


In [ ]:
# merge movies & credits
movies=movies.merge(credits,on='title')

In [ ]:
#selecting only important columns
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
#convert string - list(Data cleaning)
def convert(text):
  L=[]
  # Ensure text is a string and handle potential parsing errors
  if isinstance(text, str):
    try:
      for i in ast.literal_eval(text):
        if 'name' in i: # Use 'name' (lowercase) instead of 'Name'
          L.append(i['name'])
    except (ValueError, SyntaxError):
      return [] # Return empty list if parsing fails for this item
  return L


movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert) # Corrected column name from 'keyword' to 'keywords'
movies['cast']=movies['cast'].apply(lambda x: convert(x)[:3])

In [ ]:
#extract director name
def fetch_director(text):
  for i in ast.literal_eval(text):
    if i['job']=='diractor':
      return i['name']
  return ''

  movies['crew']=movies['cres'].apply(fetch_director)

In [ ]:
# handle overview
movies['overview'] = movies['overview'].fillna('')
movies['overview'] = movies['overview'].apply(lambda x: x.split())

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [ ]:
#finel data
new_df = movies[['movie_id','title','tags']]

In [ ]:
#vectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
#cosine similaraty
similarity = cosine_similarity(vectors)

In [ ]:
#recomandation functions
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),
                          reverse=True,
                          key=lambda x: x[1])[1:6]

    print("Recommended Movies:")
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
#test the model
recommend("The Dark Knight")

Recommended Movies:
The Dark Knight Rises
Batman Begins
Batman & Robin
Batman Forever
Batman Returns
